# Preprocessing

Preprocessing will revolve around the following steps: 
1. Draft Dataset 
- Imputation of missing variables 
- Standardisation of attirbutes
- Setting variable types
- setting positions

The alternate method used here is differend joining keys. 

Instead of using a players first registered club after 2013, we will keep all club history, with respectve jumper numbers, and 
use these additionally to attept to solve issues identified below. These mostly derive from the issue of duplicate names

This current notebook works, but has a few bugs/cases to fix such as:
- Playing no games for the first club, and some for the second 
- Players who played in 2010-2012, but not in 2013 (the cut off year)
- Changed clubs in 2013 and played no more games 
- Probably more

In [542]:
import pandas as pd
import re
# Load the datasets
draft_data = pd.read_csv('../data/raw/draft_history.csv',  index_col=[0])
position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])
player_stats = pd.read_csv('../data/raw/player_statistics.csv',  index_col=[0])
afltable_hist = pd.read_csv('../data/raw/afltables_draft.csv',  index_col=[0])

# Constant Variables

# This refers to the year of which player stats were first collected
STATISTICS_COLLECTION_YEAR = '2010'
# This refers to the year of which draft history was first captured 
DRAFT_START_YEAR = '2013'

/tmp/ipykernel_278/714907168.py:5: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])


In [543]:
# Before anything, we need to join the afltable_hist and draft_data to get all the attributes we desire

# This function standardises team names to be used as a attribute to join the 2 different draft datasets
from optparse import Values


def team_standardise(team):
    team_dict = {'Adelaide' : 'Adelaide Crows',
                  'GWS'  :  'GWS Giants',
                  'Gold Coast' : 'Gold Coast Suns',
                  'Geelong' : 'Geelong Cats',
                  'Sydney' : 'Sydney Swans',
                  'West Coast' : 'West Coast Eagles'}
    if team in team_dict.keys():
        team = team_dict[team]
    return team
    
# This function determines the last playing year of a player
def last_season(seasons):
    seasons = re.sub(r'[ |\-|,|A-z]', '', seasons)
    # take last 4 didgets
    return seasons[-4:]

# This function determines the first playing year of a player
def first_season(seasons):
    seasons = re.sub(r'[ |\-|,|A-z]', '', seasons)
    # take last 4 didgets
    return seasons[:4]

# This finds the last year a player plays over their whole career
def players_last_year(player, HT, WT, dataset = afltable_hist):
    # frst take subset with player information
    player_draft_sub = dataset[(dataset['Player'] == player) & (dataset['HT'] == HT) & (dataset['WT'] == WT)] 
    # if there is only one row, just output the club
    most_recent_year = '0'
    for history in player_draft_sub['Seasons']:
        # work out if 2013 lies before the last year
        years = last_season(history)
        if most_recent_year < years:
            most_recent_year = years
    return most_recent_year

# ensure the names are standardised to ensure compadability between datasets
def name_cleanse(name):
    # need to remove the middle names from players
    name = re.sub(' [A-z]\. ', ' ', name)
    # need to keep the first 2 letters of the first name
    first_n, last_n = name.split(' ', 1)
    # replace all (tom)s with thomas
    first_n = re.sub('[\W_| ]+', '', first_n.lower())
    if first_n in ('tom', 'tommy'):
        first_n = 'thomas'
    f_len = len(first_n)
    cap = min(f_len, 3)
    last_n = re.sub('[\W_| ]+', '', last_n.lower())
    return first_n[0:cap] + ' ' + last_n

# this finds the last item in the list
def last_item(x, current):
    if len(x) == 1:
        return x[0]
    else:
        # find the club before
        x.reverse()
        for i in x:
            if i != current:
                return i
        # if this doesn't work
        return current

# THis fuction is designed to identify unjoined entries from the afltables dataset
def chiral_leftovers(primary, secondary): 
    # This is traditional method with all unlinked removed 
    full = pd.merge(primary, secondary,  how='inner', left_on=['Player_alt','Chiral_Key'], right_on = ['Player_alt','Chiral_Key'])
    # next, create secondary with all unlinked 
    unlinked = primary[~primary['providerId'].isin(full['providerId'].values.tolist())]
    # now need to join using the second method, which has beed derived by identifying they have all joined a club and not played
    unlinked['Chiral_Key'] = unlinked.apply(lambda x: last_item(x['Teams'], x['Chiral_Key']), axis = 1)
    # now try again 
    alt_link = pd.merge(unlinked, secondary,  how='left', left_on=['Player_alt','Chiral_Key'], right_on = ['Player_alt','Chiral_Key'])
    # finally join the 2 datasets 
    dataset = pd.concat([full, alt_link], axis=0)
    return dataset



In [544]:
# First remove players whom played only in seasons before and including 2010 (want at least a year to compare to: 2010 and 2011)
afltable_hist['Last_season_club'] = afltable_hist['Seasons'].apply(lambda x: last_season(x))
afltable_hist['First_season_club'] = afltable_hist['Seasons'].apply(lambda x: first_season(x))
# remove players who finished at a club before 1995
afltable_hist = afltable_hist[afltable_hist['First_season_club'] > '1990']
# The reason we need to use the HT and WT as input, is to differenciate between players of the same name
afltable_hist['Last_season'] = afltable_hist.apply(lambda x: players_last_year(x['Player'], x['HT'], x['WT']), axis=1)
# Want only players who are registered to an afl club from 2013 onwards
afltable_hist = afltable_hist[afltable_hist['Last_season'] >= DRAFT_START_YEAR]
afltable_hist['Team'] = afltable_hist['Team'].apply(lambda x: team_standardise(x))
afltable_hist['Player_alt'] = afltable_hist['Player'].apply(lambda x: name_cleanse(x))
# for the join club
afltable_hist = afltable_hist.reset_index()
# sort again
afltable_hist.sort_values('Last_season_club', inplace=True)

/tmp/ipykernel_278/1041434780.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afltable_hist['Last_season'] = afltable_hist.apply(lambda x: players_last_year(x['Player'], x['HT'], x['WT']), axis=1)


In [545]:
# Now lets obtain the desired aggreagated statistics
# Each player can be uniquely identified by their name, height and weight
afltable_agg = afltable_hist.groupby(['Player_alt', 'HT', 'WT']).agg(Games = pd.NamedAgg('Games', 'sum'), 
                                                  Clubs = pd.NamedAgg('Team', aggfunc=lambda x: list(x)), 
                                                  Numbers = pd.NamedAgg('#', aggfunc=lambda x: list(x)),
                                                  Total_Clubs = pd.NamedAgg('Team', aggfunc=lambda x: len(list(x))),
                                                  Goals = pd.NamedAgg('Goals', 'sum'), 
                                                  Debut_season = pd.NamedAgg('First_season_club', 'min')).reset_index()

In [546]:
team_keys = afltable_agg.apply(lambda x: list(zip(x['Clubs'], x['Numbers'])), axis=1)
afltable_agg['Team_Keys'] = team_keys
chiral_keys = afltable_agg.apply(lambda x: list(zip(x['Clubs'], x['Numbers']))[-1][0], axis=1)
afltable_agg['Chiral_Key'] = chiral_keys

In [547]:
# Before anything, we need to join the afltable_hist and draft_data to get all the attributes we desire
# get common name attribute
draft_data['Player'] = draft_data['firstName'] + ' ' + draft_data['surname']
draft_data['Player_alt'] = draft_data['Player'].apply(lambda x: name_cleanse(x))
# need to setup Key_club (which is the first club a player is played at since 2013)
draft_data.sort_values(by=['season'], inplace=True)
draft_data['Team_Keys'] = draft_data.apply(lambda x: (x['team'], x['jumperNumber']), axis=1)

In [548]:
# could modify draftYear, draftType to take list, so clubs redrafted can be identified
# Each player can be uniquely identified by their id
draft_data_agg = draft_data.groupby('id').agg(Player_alt = pd.NamedAgg('Player_alt', lambda x: pd.Series.mode(x)[0]),
                                                                    player_name = pd.NamedAgg('Player', lambda x: pd.Series.mode(x)),
                                                                    first_reg_season = pd.NamedAgg('season', 'min'),
                                                                    Last_reg_season = pd.NamedAgg('season', 'max'),
                                                                    Position = pd.NamedAgg('position', lambda x: set(x)),
                                                                    providerId = pd.NamedAgg('providerId', lambda x: pd.Series.mode(x)[0]),
                                                                    dateOfBirth = pd.NamedAgg('dateOfBirth', lambda x: pd.Series.mode(x)[0]),
                                                                    draftYear = pd.NamedAgg('draftYear', lambda x: list(x)[0]),
                                                                    heightInCm = pd.NamedAgg('heightInCm', lambda x: pd.Series.mode(x)[0]),
                                                                    weightInKg = pd.NamedAgg('weightInKg', lambda x: pd.Series.mode(x)[0]),
                                                                    recruitedFrom = pd.NamedAgg('recruitedFrom', lambda x: list(x)[0]),
                                                                    debutYear = pd.NamedAgg('debutYear', lambda x: list(x)[0]),
                                                                    draftType = pd.NamedAgg('draftType', lambda x: list(x)[0]),
                                                                    draftPosition = pd.NamedAgg('draftPosition', lambda x: list(x)[0]),
                                                                    Team_Keys_1 = pd.NamedAgg('Team_Keys', lambda x: set(x)),
                                                                    Teams = pd.NamedAgg('team', lambda x: list(x)),
                                                                    Chiral_Key = pd.NamedAgg('team', lambda x: list(x)[-1])
                                                                    ).reset_index()

In [549]:
# identify the unlinked and join them with the secondary method
full_draft = chiral_leftovers(draft_data_agg, afltable_agg)

/tmp/ipykernel_278/1345496199.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unlinked['Chiral_Key'] = unlinked.apply(lambda x: last_item(x['Teams'], x['Chiral_Key']), axis = 1)


The joining of the 2 draft datasets has been completed, where we can now begin the following:
- Filling missing values
- breaking down the position and history attributes
- create new features (no. of seasons)

# Imputation

In [ ]:
# We can start by removing the HT, WT, Team_Keys, Team_Keys, Numbers from the dataset, as they are irrelevant
full_draft.drop(columns=['HT', 'WT', 'Numbers', 'Team_Keys'], inplace=True)

In [552]:
full_draft.columns[full_draft.isnull().any()]

Index(['draftYear', 'recruitedFrom', 'debutYear', 'draftType', 'draftPosition',
       'Games', 'Clubs', 'Total_Clubs', 'Goals', 'Debut_season'],
      dtype='object')